In [1]:
!pip install python-barcode
!pip install check-digit-EAN13

  Obtaining dependency information for python-barcode from https://files.pythonhosted.org/packages/10/27/9b5c5bb1938d4e6b12f4c95f40ea905c11df3cd58e128e9305397b9a2697/python_barcode-0.15.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for check-digit-EAN13: filename=check_digit_EAN13-0.2-py3-none-any.whl size=3395 sha256=b6c83c4a6adda1508a3a06aa92ce032b8b366767e4564e4370f3aa0aac5811f0
  Stored in directory: /root/.cache/pip/wheels/28/39/0b/0272c58015e177a2303ba447d784efec4d356964d3d3e02890
Successfully built check-digit-EAN13


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D, ZeroPadding2D, Lambda, Reshape, Flatten, Dense, concatenate
from tensorflow.compat.v1.image import resize_nearest_neighbor, resize_bilinear
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Lambda
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras.callbacks import *
from functools import reduce
from PIL import Image
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
import cv2
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.utils import Sequence
import os 
import cv2
from barcode import EAN13, ISBN13
from barcode.writer import ImageWriter
from tqdm import tqdm
from check_digit_EAN13.check_digit import get_check_digit

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 320
BATCH_SIZE = 256
count = 30000
path = "dataset_2"

In [4]:
if not os.path.exists(os.path.join(path, "with_numbers", "images")):
    os.makedirs(os.path.join(path, "with_numbers", "images"))
if not os.path.exists(os.path.join(path, "without_numbers", "images")):
    os.makedirs(os.path.join(path, "without_numbers", "images"))

In [5]:
for i in tqdm(range(count)):
    random_code = np.random.randint(999999999999, 9999999999999, dtype=np.int64)
    random_code = get_check_digit(random_code)
    with open("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), "wb") as file:
        EAN13(str(random_code), writer=ImageWriter()).write(file)
    # if i % 2 == 0:
    image = cv2.imread("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), cv2.IMREAD_GRAYSCALE)
    image = np.asarray(image)[2:199, 63:460]
    image = cv2.resize(image, (320, 160))
    cv2.imwrite("{}/without_numbers/images/{}_{}_0.jpg".format(path, str(random_code), str(i)), image)

100%|██████████| 30000/30000 [06:05<00:00, 82.11it/s]


In [6]:
class Datagenerator(Sequence):
    def __init__(self, image_set_path, batch_size):
        self.image_set_path = image_set_path
        self.batch_size = batch_size
        self.image_list = os.listdir(image_set_path)
        print(len(self.image_list))

    def __getitem__(self, index):
        batch_input_list = self.image_list[self.batch_size * index: self.batch_size * (1 + index)]
        
        X = np.zeros((self.batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 1))
        Y = np.zeros((self.batch_size, 13, 10))
        for i, image_name in enumerate(batch_input_list):
            image = cv2.imread(os.path.join(self.image_set_path, image_name), cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
            image = np.expand_dims(image, -1)
            image = image / 128
            X[i] = image
            Y[i] = self.encode_label(image_name.split("_")[0])            
        return X, Y

    def __len__(self):
        return len(self.image_list) // self.batch_size
    
    def encode_label(self, label):
        OHE = np.zeros((13, 10))
        for i, num in enumerate(label):
            num = int(num)
            OHE[i][num] = 1
        return OHE

In [7]:
def custom_loss_tf(y_true, y_pred):
    y_true = tf.reshape(y_true, (-1, 13, 10))
    y_pred = tf.reshape(y_pred, (-1, 13, 10))
    soft_pred = tf.math.softmax(y_pred, axis=-1)
    cross_entropy = tf.losses.categorical_crossentropy(y_true=y_true, y_pred=soft_pred)
    sum_loss = tf.math.reduce_sum(cross_entropy)
    return sum_loss

In [8]:
def Conv2D_block(inputs, out_channels, kernel_size=(1, 1), strides=(1, 1), padding="same"):
    conv = Conv2D(out_channels, kernel_size, strides=strides, padding=padding, use_bias=False)(inputs)
    bn = BatchNormalization()(conv)
    relu_out = ReLU()(bn)

    return relu_out

In [9]:
def Bottleneck(out_channels, shortcut=True, group=1, e=0.5, inputs=None):
    if shortcut:
        c_ = int(out_channels * e)
        x1 = Conv2D_block(out_channels=c_, inputs=inputs)
        x2 = Conv2D_block(out_channels=out_channels, inputs=x1, kernel_size=(3, 3), strides=(1, 1))
        add = Add()([inputs, x2])
        add = ReLU()(add)
        return add
    else:
        c_ = int(out_channels * e)
        x1 = Conv2D_block(out_channels=c_, inputs=inputs)
        x2 = Conv2D_block(out_channels=out_channels, inputs=x1, kernel_size=(3, 3), strides=(1, 1))
        return x2

In [10]:
def BottleneckCSP(out_channels, number=1, shortcut=True, group=1, e=0.5, inputs=None):
    c_ = int(out_channels * e)
    x1 = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)

    for _ in range(number):
        x1 = Bottleneck(out_channels=c_, shortcut=shortcut, e=1.0, group=group, inputs=x1)
    y2 = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)

    c = Concatenate()([x1, y2])
    c = Conv2D_block(out_channels=2 * c_, kernel_size=(1, 1), strides=(1, 1), inputs=c)
    return c

In [11]:
def SPP(c1, c2, k=(5, 9, 13), inputs=None):
    c_ = c1 // 2  
    x = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)
    x = Conv2D_block(out_channels=c2, kernel_size=(1, 1), strides=(1, 1), inputs=x)
    return x

In [12]:
def Dense_layer(num_class, inputs=None):
    if len(inputs.shape) > 2:
        inputs = Flatten()(inputs)

    dense = Dense(num_class)(inputs)

    return dense

In [13]:
def _create_model():
    input_layer = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 1),
                            name="input")

    x0 = Conv2D_block(inputs=input_layer, out_channels=16, kernel_size=(3,3), strides=(2, 2))
    x1 = Conv2D_block(inputs=x0, out_channels=32, kernel_size=(3, 3), strides=(2, 2))
    x2 = BottleneckCSP(inputs=x1, number=1, out_channels=32, shortcut=True, group=1, e=0.5)
    x3 = Conv2D_block(inputs=x2, out_channels=64, kernel_size=(3, 3), strides=(2, 2))
    x4 = BottleneckCSP(inputs=x3, number=2, out_channels=64, shortcut=True, group=1, e=0.5)
    x5 = Conv2D_block(inputs=x4, out_channels=128, kernel_size=(3, 3), strides=(2, 2))
    x6 = BottleneckCSP(inputs=x5, number=3, out_channels=128, shortcut=True, group=1, e=0.5)    
    x7 = Conv2D_block(inputs=x6, out_channels=256, kernel_size=(3, 3), strides=(1, 1))
    x9 = BottleneckCSP(inputs=x7, number=1, out_channels=256, shortcut=True, group=1, e=0.5)
    x111 = SPP(64, 32, inputs=x9)

    # block 1
    x10 = Conv2D_block(inputs=x111, out_channels=32, kernel_size=(3, 3), strides=(2, 2))
    x10 = Dense_layer(130, x10)
    
    model = Model(input_layer, x10)
    
    return model

In [14]:
model = _create_model()
model.summary()
model.compile(optimizer="adam", loss=custom_loss_tf, metrics=custom_loss_tf, run_eagerly=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 160, 320, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 80, 160, 16)          144       ['input[0][0]']               
                                                                                                  
 batch_normalization (Batch  (None, 80, 160, 16)          64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 80, 160, 16)          0         ['batch_normalization[0][0

In [15]:
data_generator = Datagenerator("/kaggle/working/dataset_2/without_numbers/images", BATCH_SIZE) 
callbacks_ckpt = ModelCheckpoint("model_{epoch:03d}.h5", save_freq=10)
image_batch_shape = (BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, 1)
label_batch_shape = (BATCH_SIZE, 13, 10)
tf_train_generator = tf.data.Dataset.from_generator(lambda: map(tuple, data_generator), 
                                                    (tf.float32, tf.float32),
                                                    (image_batch_shape, label_batch_shape))
tf_train_generator = tf_train_generator.prefetch(buffer_size=tf.data.AUTOTUNE).cache()

30000


In [16]:
tf.config.run_functions_eagerly(True)
model.fit(tf_train_generator, epochs=70, callbacks=[callbacks_ckpt], verbose=1)
model.save("last_model.h5")

Epoch 1/70
      9/Unknown - 8s 525ms/step - loss: 8122.6528 - custom_loss_tf: 8122.6528

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


117/117 [==============================] - 67s 541ms/step - loss: 3207.8750 - custom_loss_tf: 3207.8750
Epoch 2/70
117/117 [==============================] - 52s 445ms/step - loss: 77.8751 - custom_loss_tf: 77.8751
Epoch 3/70
117/117 [==============================] - 52s 443ms/step - loss: 21.7415 - custom_loss_tf: 21.7415
Epoch 4/70
117/117 [==============================] - 52s 443ms/step - loss: 11.4298 - custom_loss_tf: 11.4298
Epoch 5/70
117/117 [==============================] - 51s 439ms/step - loss: 7.1930 - custom_loss_tf: 7.1930
Epoch 6/70
117/117 [==============================] - 52s 443ms/step - loss: 4.9809 - custom_loss_tf: 4.9809
Epoch 7/70
117/117 [==============================] - 51s 439ms/step - loss: 3.6647 - custom_loss_tf: 3.6647
Epoch 8/70
117/117 [==============================] - 52s 441ms/step - loss: 2.8123 - custom_loss_tf: 2.8123
Epoch 9/70
117/117 [==============================] - 52s 444ms/step - loss: 2.2262 - custom_loss_tf: 2.2262
Epoch 10/70
117/11

In [17]:
path = "test"
if not os.path.exists(os.path.join("test", "with_numbers", "images")):
    os.makedirs(os.path.join("test", "with_numbers", "images"))
if not os.path.exists(os.path.join("test", "without_numbers", "   images")):
    os.makedirs(os.path.join("test", "without_numbers", "images"))
count = 30
for i in tqdm(range(count)):
    random_code = np.random.randint(999999999999, 9999999999999, dtype=np.int64)
    random_code = get_check_digit(random_code)
    with open("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), "wb") as file:
        EAN13(str(random_code), writer=ImageWriter()).write(file)
    # if i % 2 == 0:
    image = cv2.imread("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), cv2.IMREAD_GRAYSCALE)
    image = np.asarray(image)[2:199, 63:460]
    image = cv2.resize(image, (320, 160))
    cv2.imwrite("{}/without_numbers/images/{}_{}_0.jpg".format(path, str(random_code), str(i)), image)

100%|██████████| 300/300 [00:03<00:00, 85.11it/s]


In [18]:
def filter_predicitons_tf(y_pred):
    barcode_list = []
    y_pred = np.reshape(y_pred, (-1, 13, 10))
#     print(tf.make_ndarray(y_pred))
    pred_idx = np.argmax(y_pred, axis=-1)
    for pred_barcode in pred_idx:
        barcode = ""
        for num in pred_barcode:
            barcode += str(num)
        barcode_list.append(barcode)
    return barcode_list

def humming_distance(string_1, string_2):
    i = 0
    count = 0
    while i < len(string_1):
        if string_1[i] != string_2[i]:
            count += 1
        i += 1
    return count

In [19]:
res_count = 0
for image_name in os.listdir("/kaggle/working/test/without_numbers/images"):
    image = cv2.imread("/kaggle/working/test/without_numbers/images/"+ image_name, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    image = np.expand_dims(image, -1)
    image = image / 128
    image = np.asarray([image])
    res = model.predict([image])
    res = filter_predicitons_tf(res)
#     print(image_name.split("_")[0], res[0])
    dist = humming_distance(image_name.split("_")[0], res[0])
    if dist == 0:
        res_count = res_count + 1
    print(image_name, " : ", res[0], " : ", dist)

/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 258ms/step
6190328652884_285_0.jpg  :  6190328652884  :  0
1/1 [==============================] - 0s 83ms/step
7715549180450_121_0.jpg  :  7715549180450  :  0
1/1 [==============================] - 0s 84ms/step
5182578540142_0_0.jpg  :  5182578540142  :  0
1/1 [==============================] - 0s 85ms/step
8653773412376_39_0.jpg  :  8653773412376  :  0
1/1 [==============================] - 0s 86ms/step
4821317071547_139_0.jpg  :  4821317071547  :  0
1/1 [==============================] - 0s 86ms/step
2195916935251_40_0.jpg  :  2195916935251  :  0
1/1 [==============================] - 0s 88ms/step
7363760614171_191_0.jpg  :  7363760614171  :  0
1/1 [==============================] - 0s 89ms/step
2550492630479_171_0.jpg  :  2550492630479  :  0
1/1 [==============================] - 0s 85ms/step
5836141563679_50_0.jpg  :  5836141563679  :  0
1/1 [==============================] - 0s 84ms/step
4980833482715_272_0.jpg  :  4980833482715  :  0
1/1 